In [60]:
!git clone https://github.com/PonyGE/PonyGE2.git

fatal: destination path 'PonyGE2' already exists and is not an empty directory.


In [61]:
!cd PonyGE2/ && git pull

Already up to date.


In [62]:
#pip installs

%pip install numpy

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: numpy in c:\users\hsabi\appdata\local\programs\python\python311\lib\site-packages (1.24.1)




[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [63]:
%pip install sewar



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
%pip install openpyxl


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
%%capture captured
import shutil
import os

experiment = "regression"

shutil.copy(f"./fitness/{experiment}.py", "./PonyGE2/src/fitness/")
!cd ./PonyGE2/src && python ponyge.py --parameters ../../parameters/{experiment}.txt
os.remove(f"./PonyGE2/src/fitness/{experiment}.py")


In [54]:
import numpy as np
from sewar.full_ref import mse
from openpyxl import load_workbook

captured_str = str(captured).split('\n')
for line in captured_str:
    if "Phenotype: " in line:
        phenotype = line.replace("Phenotype: ", "")
    elif "DATASET_NAME" in line:
        dataset_name = line.replace("DATASET_NAME: ", "")

dataset_path = './datasets/data.xlsx'



In [55]:
import torch
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def extremeVal(reference, val):
    if (val > 1e2*reference) or (val < -1e2*reference):
        return 0
    else:
        return val

def load_data():
    wb = load_workbook(dataset_path)
    dataset_name = ["CARSALES", "LAKEERIE", "GAS", "Electricity",
                    "PIGS", "POLLUTION", "REDWINE", "SUNSPOT", "Nordic", "B1H"]
    dataset = {}
    for name in dataset_name:

        ws = wb[name]
        columns = [ws["A"], ws["B"], ws["C"],
                   ws["D"], ws["E"], ws["F"], ws["G"]]

        sheet = [[] for _ in range(len(columns[0]))]

        for c in columns:
            for index, item in enumerate(c):
                sheet[index].append(item.value)

        sheet = np.array(sheet)
        dataset[name] = {"raw": sheet}

    for name, _ in dataset.items():
        sheet = dataset[name]["raw"]

        dataset[name]["series"] = {
            "treino": [], "teste": [], "validacao": []}
        dataset[name]["arima"] = {
            "treino": [], "teste": [], "validacao": []}
        dataset[name]["mlp"] = {"treino": [], "teste": [], "validacao": []}
        dataset[name]["svr"] = {"treino": [], "teste": [], "validacao": []}
        dataset[name]["rbf"] = {"treino": [], "teste": [], "validacao": []}

        for row in sheet[1:]:
            split_type = row[2]
            if row[4] == None:
                continue
            key = None
            if split_type == "Teste":
                key = "teste"
            elif split_type == "Validacao":
                key = "validacao"
            elif split_type == "Treinamento":
                key = "treino"
            else:
                continue

            dataset[name]["series"][key].append(float(row[1]))
            dataset[name]["arima"][key] .append(float(row[3]))
            dataset[name]["mlp"][key]   .append(float(row[4]))
            dataset[name]["svr"][key]   .append(float(row[5]))
            dataset[name]["rbf"][key]   .append(float(row[6]))

        for type_data in ["teste", "treino", "validacao"]:
            dataset[name]["series"][type_data] = np.array(
                dataset[name]["series"][type_data])
            dataset[name]["arima"][type_data] = np.array(
                dataset[name]["arima"][type_data])
            dataset[name]["mlp"][type_data] = np.array(
                dataset[name]["mlp"][type_data])
            dataset[name]["svr"][type_data] = np.array(
                dataset[name]["svr"][type_data])
            dataset[name]["rbf"][type_data] = np.array(
                dataset[name]["rbf"][type_data])

        fixExtreme = np.vectorize(extremeVal)

    for name in dataset.keys():
        reference = np.max(dataset[name]["series"]["treino"])

        for type_data in ["teste", "treino", "validacao"]:
            dataset[name]["mlp"][type_data] = fixExtreme(
                reference, dataset[name]["mlp"][type_data])
            dataset[name]["svr"][type_data] = fixExtreme(
                reference, dataset[name]["svr"][type_data])
            dataset[name]["rbf"][type_data] = fixExtreme(
                reference, dataset[name]["rbf"][type_data])
            dataset[name]["arima"][type_data] = fixExtreme(
                reference, dataset[name]["arima"][type_data])

    return dataset[name]

def build_model(phenotype):
    model = {"linear": {}, "nlinear": {}}

    linear, nlinear = phenotype.split(';')

    weight, linear_model = linear.split(':')
    model["linear"][linear_model] = float(weight)

    nlinear_parts = nlinear.split(',')

    for i in range(len(nlinear_parts)):
        weight, nlinear_model = nlinear_parts[i].split(':')
        model["nlinear"][nlinear_model] = float(weight)

    return model

def predict_mse(dataset, model, series_name):
    predict = dataset["series"][series_name]
    window_size = 1
    X_train = apply_window(dataset,
        window_size, series_name, model_names=model["nlinear"].keys())
    kernel = []
    for _ in range(window_size):
        for _, val in model["linear"].items():
            kernel.append(val)
        for _, val in model["nlinear"].items():
            kernel.append(val)
    return mse(create_prediction(X_train, np.reshape(kernel, (1, -1))), predict)

def apply_window(dataset, window_size, data_type, model_names=None):
    size_pred = len(dataset["series"][data_type])
    new_data = []
    for w in range(window_size):
        linear = np.concatenate(
            [np.zeros(w), dataset["arima"][data_type]])
        linear = linear[:size_pred]
        row = np.array(linear)
        for model_name in model_names:
            nonlinear = np.concatenate(
                [np.zeros(w), dataset[model_name][data_type]])
            nonlinear = nonlinear[:size_pred]
            row = np.column_stack([row, nonlinear])
        if len(new_data) == 0:
            new_data = row
        else:
            new_data = np.column_stack([new_data, row])
    return np.array(new_data)

def create_prediction(X, kernel):
    X = torch.tensor(np.expand_dims(X, axis=(0, 1))).float()
    kernel = torch.tensor(np.expand_dims(kernel, axis=(0, 1))).float()
    X.to(device)
    kernel.to(device)
    result = F.conv2d(X, kernel)
    return result.numpy().squeeze().astype("f")


In [56]:
dataset = load_data()
model = build_model(phenotype)
print(f'Dataset: {dataset_name}')
print(f'Fenótipo: {phenotype}')
print('Fitness treino:', f"{predict_mse(dataset, model, 'treino'):.3e}")
print('Fitness teste:', f"{predict_mse(dataset, model, 'teste'):.3e}")
print('Fitness validacao:', f"{predict_mse(dataset, model, 'validacao'):.3e}")


Dataset: B1H
Fenótipo:   000.096:arima;1.2:mlp
Fitness treino: 2.157e-02
Fitness teste: 2.102e-02
Fitness validacao: 8.845e-03
